## Affine&Softmax层的实现

### Affine层

这里的affine层其实就是说神经网络中的，信号加权的计算过程，也就是XW+B这个公式的计算。

> Affine表示仿射的意思，这是几何的概念，仿射一般是指对图形的平移或者线性伸缩。也就对应这里的对于输入信号的权重的线性变换和偏置的平移变换。

如下coding

In [4]:
import numpy as np

# 输入
X = np.random.rand(2)
# 权重 W
W = np.random.rand(2,3)
# 偏置值 B
B = np.random.rand(3)

print(X.shape,X)
print(W.shape,W)
print(B.shape,B)

Y = np.dot(X,W) + B
print(Y.shape,Y)

(2,) [0.03522538 0.33746387]
(2, 3) [[0.71601237 0.59287577 0.8510641 ]
 [0.57232106 0.74278995 0.62142448]]
(3,) [0.68368357 0.8806054  0.13857277]
(3,) [0.90204306 1.15215445 0.37826014]


从上面的例子中我们可以看到的数据是,以上的数据都是向量或者矩阵。Y是经过矩阵的点乘获得的，其矩阵变换如下：
![](imgs/19.jpg)

计算图如下：
![](imgs/20.jpg)

上面所有的数据都是多维数组的格式。

反向传播的过程中，对于向量的点乘t运算，其实这个也就是相当于标量的乘法求导，这里要注意的是：
- 向量求导要注意两个数据的位置
- 其次是要注意需要转置
计算图如下：
![](imgs/21.jpg)

这里有一个推导的过程如下：
![](imgs/22.jpg)

### 批量数据集的Affine层

上面的公式是单个数据集输入的时候反向传播的过程层。现在来看看，如果是批量数据我们又将如何来处理。
![](imgs/23.jpg)

对比刚刚上面的变换，dW的shape是没有任何变化的，主要的变换是在X的数据变成了N,这样反向传播过来的数据也在这个维度数量上变成了N。

上面看到关于dW的权重的计算过程和公式，现在来分析一下dB是如何计算的。
B正向传播时，会根据批量数据的个数N，利用广播机制将数据自动扩展在对于的数据个数N。
dB反向传播时，会将这N个数据汇总到偏置的元素上，因为要保证dB的shape与B统一。这里的汇总其实就是**对N个数据的导数按元素进行求和**
先看一个coding的例子，大家来感受一下：


In [3]:
import numpy as np
# 这个是X.W的结果
XdotW = np.array([[0,0,0],[10,10,10]])
B = np.array([1,2,3])
# 下面需要计算加上偏置值
Y = XdotW + B
# 这里虽然B是是个一维的向量
# 但是numpy会利用广播机制
# 将B扩展为（2，3）
# np.array([[1,2,3],[1,2,3]])
print(Y)

dY = np.array([[1,2,3],[4,5,6]])
# axis=0,对第0个轴方向上的元素进行求和
dB = np.sum(dY,axis=0)
print(dB)

[[ 1  2  3]
 [11 12 13]]
[5 7 9]


In [6]:
# 批量数据Affine层的class

class Affine:
    def __init__(self,W,b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self,x):
        self.x = x
        out = np.dot(x,self.W) + b
        return out
    
    def backward(self,dout):
        dx = np.dot(dout,self.W.T)
        self.dW = np.dot(self.x.T,dout)
        self.db = np.sum(dout,axis=0)
        
        return dx

### sofmax-with_Loss 层

前面几个小节中我们看到了关于激活函数反向传播（sigmoid/ReLU）以及刚刚看到的Affine层也就时全连接层的反向传播中计算的(dW,db),现在我们来看看组成神经网络的最后一层时如何进行反向传播的。
比如分类的话，我们一般使用softmax来进行概率评估。
如下图所示：
![](imgs/24.jpg)

这里我们要注意两个概念：推理与学习

>深度学习中的推理（inference）如果是分类问题，是给一个数据来判断该数据是属于哪一类，如果是回归问题，是给一个数据来预测该数据的结果是多少。所以这个过程只需要给出结果即可。一般是在神经网络最后一层中获得输出中获得一个结果。所以这里可能不需要计算softmax层，直接从Affine层给出一个答案即可。

>深度学习中的学习（training）这里的学习就是训练，这个时候我们不仅需要最后一层给出结果，也需要给出具体与答案差多少。这个时候就需要softmax层对输出的做一个正规化，将结果进行概率评分，这样方便我们下一步的反向传播，来调整参数使得模型更加的符合我们预期的输出。

好了上面介绍学习过程，这里需要我们给出与预期值的误差。这个时候我们就需要损失函数了。我们先从cross entropy error来分析。交叉熵误差是我们经常要用的loss function。该函数的计算图如下:

![](imgs/25.jpg)

上面的有点复杂，下面来看看简化版本
![](imgs/26.jpg)

在softmax-loss反向传播最终得到的结果是$(y_1-t_1,y_2-t_2,y_3-t_3)$ 这个不就是训练值与目标值的误差吗？这样通过这个误差在反向传播来修改和更新对应的参数。

下面coding如下：



In [1]:
# softmax 函数的改进
import numpy as np
def softmax(a):
    c= np.max(a)
    b = a - c # 防止溢出
    exp_a = np.exp(b)
    sum_exp_a = np.sum(exp_a)
    y = exp_a/sum_exp_a
    return y

In [2]:
def cross_entropy_error(y,t):
    # 这里引入reshape变换
    #为了方便计算batch_size的输入
    if y.ndim == 1:
        t = t.reshape(1,t.size)
        y = y.reshape(1,y.size)
    batch_size = y.shape[0]
    return -np.sum(t*np.log(y+1e-7))/batch_size


In [3]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self,x,t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y,self.t)
        
    def backward(self,dout=1):
        #获取批量数据的大小
        batch_size = self.t.shape[0]
        # 需要平均化最后的结果
        dx = (self.y-self.t)/batch_size
        
        return dx